In [1]:
import os
import ast
import pickle
import time
from tqdm import tqdm
import json
from glob import glob
from collections import Counter, defaultdict
import sys
sys.path.insert(0, '../')

import pandas as pd
import numpy as np

from load_data import load_user_time_read, load_raw
from tfidf import TFIDFGenerator, get_df
from config import Config
from utils import iterate_data_files, squeeze, save_as_pickle
from preprocessing import PostIdEncoder

In [2]:
start = time.time()
user_time_read = load_user_time_read(root_dir='../preprocessed/user_time_read.json')
encoder = PostIdEncoder(root_dir='../encodings')
tfidf = TFIDFGenerator('../tfidf')
print(f'Loading time: {time.time() - start}')

Loading time: 17.318955183029175


In [3]:
user_list = list(user_time_read.keys())
history = user_time_read[user_list[0]] # 한 유저의 히스토리

In [4]:
# user_preferences = []
posts = pd.DataFrame()
user_preferences = np.zeros((7000, 1))
# posts = np.zeros((1, 7000))


for i in tqdm(range(10)):
    history = user_time_read[user_list[i]]
    # temp = pd.DataFrame()
    user_preference = np.zeros((1, 7000))
    for h in history:
        partial_tfidf = tfidf.generate(encoder.transform(h[-1]), drop_id=False)
        # temp = pd.concat([temp, partial_tfidf.drop('post_meta_id', axis=1)])
        user_preference += partial_tfidf.drop('post_meta_id', axis=1).values.sum(axis=0)
        posts = pd.concat([posts, partial_tfidf], axis=0, ignore_index=True)
        # posts = np.vstack([posts, partial_tfidf.drop('post_meta_id', axis=1).values])

    user_preferences = np.hstack([user_preferences, user_preference.reshape(7000, 1)])

user_preferences = user_preferences[:, 1:]
posts = posts.drop_duplicates(ignore_index=True)
post_meta_id = posts['post_meta_id'].tolist()
posts = posts.drop('post_meta_id', axis=1)
idf = np.log(tfidf.DF.values.squeeze()) - np.log((posts != 0).sum().values + 1e-4)

100%|██████████| 10/10 [00:17<00:00,  1.72s/it]


In [5]:
recommend_output = np.matmul((posts * idf).values, user_preferences)
enc = PostIdEncoder(root_dir='../encodings')

In [6]:
recommend_output_user = pd.DataFrame(dict(value=recommend_output[:, 1], id=post_meta_id)).sort_values(by='value', ascending=False)

In [8]:
recommend_output_user['id'] = recommend_output_user['id'].apply(lambda x: enc.inverse_transform(x))

In [10]:
recommend_output_user.head(100)

,value,id
130,6474.603422,@charlieoppa86_39
161,6272.859652,@tenbody_1305
139,5982.949432,@ohsukwon_17
131,5660.779012,@sosoceo_72
64,5574.078322,@hanriverkim_4
...,...,...
152,1869.986179,@hyungsukkim_97
1395,1867.406208,@thebookman_61
674,1853.928372,@yoonjikwon_208
154,1850.626101,@hyungsukkim_91
